In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

In [10]:
## First challenge: With general framework before improvements

def black_scholes_price(S,K,T,r,sigma, call = True):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma * (T**0.5))
    d2 = (d1 - sigma*T**0.5)
    if call:
        return S * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [11]:
# Testing BS function

S = 100
K = 110
T = 1
r = 0.05
sigma = 0.2

print(black_scholes_price(S,K,T,r,sigma, call = False))

10.675324824802793


In [12]:
def implied_volatility(market_price, S, K, T, r, call = True, tol =1e-6, max_iter = 1000):
    # Initial guesses
    sigma = 0.2
    for _ in range(max_iter):
        # Calculate the option price with the current sigma
        bs_price = black_scholes_price(S, K, T, r, sigma, call)
        # Derivative of Black-Scholes price with respect to sigma (vega)
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)  # PDF of d1

        # Update sigma using Newton-Raphson
        sigma_new = sigma - (bs_price - market_price) / vega
        if abs(sigma_new - sigma) < tol:
            return sigma_new
        sigma = sigma_new
    raise ValueError("Implied volatility did not converge")

In [23]:
# Testing Implied volatility

market_prices = 6

implied_vols = implied_volatility(market_price = market_prices, S=S, K=K, T=T, r=r) 

print(implied_vols)

0.19898695408537717


In [24]:
# Batch implied volatility calculation

from datetime import datetime

def calculate_time_to_maturity(expiry):
    expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
    today = datetime.now()
    return (expiry_date - today).days / 365.0
